## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,East London,ZA,-33.0153,27.9116,54.88,63,86,29.37,light rain
1,1,Cabo San Lucas,MX,22.8909,-109.9124,88.90,74,40,11.50,scattered clouds
2,2,Qaanaaq,GL,77.4840,-69.3632,40.32,88,98,7.87,overcast clouds
3,3,Pangnirtung,CA,66.1451,-65.7125,42.30,93,100,2.71,overcast clouds
4,4,Chuy,UY,-33.6971,-53.4616,58.39,87,2,9.95,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
#clean_df
filtered_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
filtered_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Ciudad Bolivar,VE,8.1222,-63.5497,78.69,82,79,2.44,broken clouds
6,6,Renqiu,CN,38.6989,116.0936,82.62,79,96,3.33,overcast clouds
8,8,Hanstholm,DK,57.1167,8.6167,65.30,79,100,10.76,overcast clouds
9,9,Bethel,US,41.3712,-73.4140,73.36,63,0,0.00,clear sky
13,13,Kapaa,US,22.0752,-159.3190,83.12,65,0,11.50,clear sky
15,15,Gamba,GA,-2.6500,10.0000,67.86,92,1,4.61,clear sky
17,17,Hilo,US,19.7297,-155.0900,84.27,63,75,8.05,broken clouds
19,19,Saint-Philippe,RE,-21.3585,55.7679,66.22,73,42,13.73,scattered clouds
20,20,Butaritari,KI,3.0707,172.7902,81.97,76,37,13.87,scattered clouds
23,23,Jamestown,US,42.0970,-79.2353,67.64,73,0,4.61,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
filtered_df.count()

City_ID                373
City                   373
Country                373
Lat                    373
Lng                    373
Max Temp               373
Humidity               373
Cloudiness             373
Wind Speed             373
Current Description    373
dtype: int64

In [6]:
filtered_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# There is no null
clean_df = filtered_df.dropna(how='any')

In [8]:
# check if there is any NULL value again
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Ciudad Bolivar,VE,78.69,broken clouds,8.1222,-63.5497,
6,Renqiu,CN,82.62,overcast clouds,38.6989,116.0936,
8,Hanstholm,DK,65.30,overcast clouds,57.1167,8.6167,
9,Bethel,US,73.36,clear sky,41.3712,-73.4140,
13,Kapaa,US,83.12,clear sky,22.0752,-159.3190,
15,Gamba,GA,67.86,clear sky,-2.6500,10.0000,
17,Hilo,US,84.27,broken clouds,19.7297,-155.0900,
19,Saint-Philippe,RE,66.22,scattered clouds,-21.3585,55.7679,
20,Butaritari,KI,81.97,scattered clouds,3.0707,172.7902,
23,Jamestown,US,67.64,clear sky,42.0970,-79.2353,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]        
    except (IndexError):
        print("Hotel not found... Skipping.")
        

Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.


In [11]:
# List all with no Hotel Name specified
hotel_df.loc[(hotel_df["Hotel Name"] == '')]

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
15,Gamba,GA,67.86,clear sky,-2.6500,10.0000,
62,Albany,US,71.06,scattered clouds,42.6001,-73.9662,
91,Bowmore,US,80.74,clear sky,34.9421,-79.3031,
93,Pangoa,PE,69.06,scattered clouds,-12.1167,-73.0000,
134,Takhtamygda,RU,72.70,scattered clouds,54.1000,123.5667,
186,Hegang,CN,73.80,clear sky,47.4000,130.3667,
193,Avera,US,78.31,clear sky,33.1940,-82.5271,
220,Shumskiy,RU,65.98,light rain,54.8318,99.1328,
242,Airai,TL,71.83,broken clouds,-8.9266,125.4092,
272,Merauke,ID,79.43,broken clouds,-8.4667,140.3333,


In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"] != '')]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Ciudad Bolivar,VE,78.69,broken clouds,8.1222,-63.5497,Hotel Edi
6,Renqiu,CN,82.62,overcast clouds,38.6989,116.0936,City Garden Renqiu Meishicheng Store
8,Hanstholm,DK,65.30,overcast clouds,57.1167,8.6167,Hotel Hanstholm
9,Bethel,US,73.36,clear sky,41.3712,-73.4140,Courtyard by Marriott Danbury
13,Kapaa,US,83.12,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
17,Hilo,US,84.27,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
19,Saint-Philippe,RE,66.22,scattered clouds,-21.3585,55.7679,Le four à pain
20,Butaritari,KI,81.97,scattered clouds,3.0707,172.7902,Isles Sunset Lodge
23,Jamestown,US,67.64,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
24,Ushtobe,KZ,79.11,clear sky,45.2514,77.9838,"Gostinitsa ""Diyar"""


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))